In [1]:
import sys
sys.path.append('/home/myk/min0/LCNN')
sys.path.append('/home/myk/min0/LSTM')
import torch
import numpy as np
import pandas as pd
import random
import preprocess as pp

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic   
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

In [3]:
x = np.load('x_triple2token_dict.npy')
y = pp.getlabel("metadata.csv")

LCNN_model = torch.load('LCNN/LCNN_ver7.pt')
LSTM_model = torch.load('LSTM/LSTM_triple2token_ver1.pt')
Glove_model = torch.load('LSTM/LSTM_GLove_ver2.pt')
LCNNwG_model = torch.load('LCNN/LCNN_withGlove_ver1.pt')

0 is arkio
1 is bigballers
2 is bigscreen
3 is blaston
4 is campfire
5 is epicrollercoasters
6 is firsthand
7 is firsttouch
8 is fitxr
9 is godsofgravity
10 is gorillatag
11 is gunraiders
12 is horizonworkrooms
13 is horizonworlds
14 is hyperdash
15 is immersed
16 is netflix
17 is roblox
18 is vrchat
19 is youtube


In [4]:
batch = 16

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,shuffle=True, random_state=1)

from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = CustomDataset(x_train,y_train)
test_dataset = CustomDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True,drop_last=True)



In [6]:
LCNN_model.eval()
LSTM_model.eval()
Glove_model.eval()
LCNNwG_model.eval()

total =0
correct=0
incorrect_num_correct = np.zeros(120)
incorrect_num_answer = np.zeros(120)
with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)
        y= label.to(device)
        output1 = LCNN_model.forward(x)
        output2 = LSTM_model.forward(x)
        output3 = Glove_model.forward(x)
        output4 = LCNNwG_model.forward(x)
        
        output=output1+output2+output3+output4
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()
        
        for i in range(len(y)):
            if y[i]!=output_index[i]:
                k = int(y[i].cpu().numpy())
                incorrect_num_correct[k] += 1
                incorrect_num_answer[int(output_index[i].cpu().numpy())]+=1

    
    # 정확도 도출
    print("Accuracy of Test Data: {}%".format(100*correct/total))

Accuracy of Test Data: 87.3397445678711%
